In [1]:
# Pull rawdata
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np

data=pd.read_csv('Final_Project/Full_Data.csv')

C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,51,53,54,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data["HR/PA"] = data["HR_bat"]/data["PA"]
data["R/PA"] = data["R_bat"]/data["PA"]
data["RBI/PA"] = data["RBI"]/data["PA"]
data.replace(np.inf, 0 , inplace=True)

In [3]:
data['MVP_Classification']=data['awardID']
data['MVP_Classification'] = data['MVP_Classification'].map({'MVP': 1})
data['MVP_Classification'] = data['MVP_Classification'].replace(np.nan, 0)

In [4]:
data_encoded = pd.get_dummies(data, columns=["Pos"])
data_encoded.head()

,Season,Name_bat,Team_bat,Age_bat,G_bat,PA,H_bat,1B,2B,3B,...,MVP_Classification,Pos_1B,Pos_2B,Pos_3B,Pos_C,Pos_CF,Pos_LF,Pos_P,Pos_RF,Pos_SS
0,1963,Buster Narum,Orioles,22.0,7.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
1,2010,Gustavo Chacin,Astros,29.0,44.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
2,1938,Bill Lefebvre,Red Sox,22.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
3,2008,Felix Hernandez,Mariners,22.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
4,2005,Eddie Rogers,Orioles,26.0,8.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [5]:
years = data_encoded[['Season']].groupby('Season').sum().reset_index()
years_bucket_1=years[(years['Season']>=1933) & (years['Season']<=1968)]
years_bucket_2=years[(years['Season']>=1969) & (years['Season']<=1983)]
years_bucket_3=years[(years['Season']>=1984) & (years['Season']<=2007)]
years_bucket_4=years[years['Season']>=2008]

In [17]:
# Create models for years_bucket_x
for i in years_bucket_1['Season']:
    print(i)
    
    league = "NL"
    columns = ['PA','HR/PA', 'R/PA', 'RBI/PA','SB',
                 'AVG','W_pit', 'L_pit', 'ERA', 'G_pit',
                 'CG', 'ShO', 'IP',
                 'K/9', 'BB/9', 'WHIP','FP','Playoffs']
    
    # -----------------------------------------------------------------------------
    # Create Random Forest
    # Create datasets
    data_name = data_encoded[(data_encoded['Season']>=(i-2)) & (data_encoded['Season']<=i)]
    data_name=data_name[data_name['lgID']==league]
    data_name = data_name.fillna(value=0)
    y_name=data_name['MVP_Classification']
    X_name_encoded=data_name[columns]

    # Create Random Forest classifier
    rfc = RandomForestClassifier(n_estimators=500)
    rfc = rfc.fit(X_name_encoded, y_name)
    
    # ------------------------------------------------------------------------------
    # Create datasets
    data_name_reg=data_name[data_name['MVP_Classification']==1]
    y_name_reg=data_name_reg['pointsWon']
    X_name_reg=data_name_reg[columns]
        
    #create random forest regressor
    rfr = RandomForestRegressor(n_estimators=500)
    rfr = rfr.fit(X_name_reg,y_name_reg)
    
    # Get feature importance
    importances = rfr.feature_importances_
    list=sorted(zip(rfr.feature_importances_, X_name_encoded.columns), reverse=True)
    # Top 10 feature importances
    list_10=pd.DataFrame(list[:10],columns=['Score','Stat'])
    list_10['Season'] = i
    list_10['lgID'] = league
    list_10.to_csv('feature_importance_1.csv',mode='a',header=False)
                                 
    for j in years['Season']:
        print(j)
        data_test = data_encoded[data_encoded['Season']==j]
        data_test=data_test[data_test['lgID']==league]
        data_test = data_test.fillna(value=0)
        X_test_encoded=data_test[columns]
                                 
        data_test['MVP Prediction'] = rfc.predict(X_test_encoded)
                                 
        test_mvps = data_test[data_test['MVP Prediction']==1]
        
        y_reg_test=test_mvps['pointsWon']
        X_reg_test=test_mvps[columns]
        
        try:
            test_mvps['MVP Points'] = rfr.predict(X_reg_test)
            test_mvps['Model Year'] = i

            test_mvps.to_csv('results_1.csv',index=False, mode='a', header=False)
        except:
            print("No prediction for ",j)

1933
1931
1932


C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
No prediction for  1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
1934
1931
1932


C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
No prediction for  1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
1935
1931
1932


...

1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
No prediction for  1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
1967
1931
1932


C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
1968
1931
1932


C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\codyb\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
No prediction for  1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
